<a href="https://colab.research.google.com/github/Skander28/Models/blob/main/bilstm_SA(similarity1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences


In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import pandas as pd
tweets_ids = pd.read_csv('/content/drive/MyDrive/maghrebi/dialect_dataset.csv')
     

# Read the given dataset which contain the Ids and the labels
df_clean = pd.read_csv('/content/drive/MyDrive/maghrebi/messages.csv',
                 lineterminator='\n')

column_names = ['id', 'tweets']  # list of column names

df_clean.columns = column_names

tweets_dataset = pd.merge(tweets_ids, df_clean, on='id', how='inner')
tweets_dataset.head()

,id,dialect,tweets
0,1175358310087892992,IQ,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .
1,1175416117793349632,IQ,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...
2,1175450108898565888,IQ,@KanaanRema مبين من كلامه خليجي
3,1175471073770573824,IQ,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐
4,1175496913145217024,IQ,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺


In [25]:
filtered_df = tweets_dataset[tweets_dataset['dialect'].isin(['TN','DZ','MA','LY'])]
filtered_df.head()

,id,dialect,tweets
15497,1009754958479151232,LY,@toha_Altomy @gy_yah قليلين ادب ومنافقين. لو ا...
15498,1009794751548313600,LY,@AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ...
15499,1019989115490787200,LY,@smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بير...
15500,1035479791758135168,LY,@AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا...
15501,1035481122921164800,LY,@lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه ا...


In [27]:
import string
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations
     

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
     

filtered_df['tweets'] = filtered_df['tweets'].apply(remove_punctuations)
filtered_df.head()

<ipython-input-27-b5b191068311>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tweets'] = filtered_df['tweets'].apply(remove_punctuations)


,id,dialect,tweets
15497,1009754958479151232,LY,tohaAltomy gyyah قليلين ادب ومنافقين لو اختهم ...
15498,1009794751548313600,LY,AlmFaisal 😂😂 الليبيين متقلبين\nبس بالنسبة ليا ...
15499,1019989115490787200,LY,smsm071990 ALMOGRBE كل 20 تانيه شاب ليبي بيرتا...
15500,1035479791758135168,LY,AboryPro lyranoo85 رانيا عقليتك متخلفة اولا ال...
15501,1035481122921164800,LY,lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه از...


In [29]:
def remove_non_arabic(text):
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = re.sub('([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+',' ',text)
    text = re.sub(r'\\u[A-Za-z0-9]\\+',' ',text)         #'\u003c\1'  error
    return text
     

filtered_df['tweets'] = filtered_df['tweets'].apply(remove_non_arabic)
filtered_df.head()

<ipython-input-29-ecc1150cd008>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tweets'] = filtered_df['tweets'].apply(remove_non_arabic)


,id,dialect,tweets
15497,1009754958479151232,LY,قليلين ادب ومنافقين لو اختهم او قريبتهم تت...
15498,1009794751548313600,LY,الليبيين متقلبين\nبس بالنسبة ليا انا ميلي...
15499,1019989115490787200,LY,كل تانيه شاب ليبي بيرتاح لبنت مختلفة ويل...
15500,1035479791758135168,LY,رانيا عقليتك متخلفة اولا الانسان يلي يحتاج...
15501,1035481122921164800,LY,شكلك متعقدة علشان الراجل لي تحبيه ازوج بنت ي...


In [37]:
import re

def remove_repeating_characters(text):
    text = text.strip()
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    return text

filtered_df['tweets'] = filtered_df['tweets'].apply(remove_repeating_characters)
filtered_df.head()

,id,tweets,dialect
0,1009754958479151232,قليلين ادب ومنافقين اختهم او قريبتهم تتعاكس تق...,LY
1,1009794751548313600,الليبيين متقلبين بالنسبه ليا انا ميليشياوي زما...,LY
2,1019989115490787200,تانيه شاب ليبي بيرتاح لبنت مختلفه ويلاحظ انها ...,LY
3,1035479791758135168,رانيا عقليتك متخلفه اولا الانسان يلي يحتاج اهل...,LY
4,1035481122921164800,شكلك متعقده علشان الراجل تحبيه ازوج بنت يتيمه ...,LY


In [38]:
import re
import nltk

In [39]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop=stopwords.words('arabic')
     
filtered_df['tweets'] = filtered_df['tweets'].apply(lambda x:" ".join(x for x in x.split() if x not in stop))
filtered_df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,tweets,dialect
0,1009754958479151232,قليلين ادب ومنافقين اختهم او قريبتهم تتعاكس تق...,LY
1,1009794751548313600,الليبيين متقلبين بالنسبه ليا انا ميليشياوي زما...,LY
2,1019989115490787200,تانيه شاب ليبي بيرتاح لبنت مختلفه ويلاحظ انها ...,LY
3,1035479791758135168,رانيا عقليتك متخلفه اولا الانسان يلي يحتاج اهل...,LY
4,1035481122921164800,شكلك متعقده علشان الراجل تحبيه ازوج بنت يتيمه ...,LY


In [40]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
filtered_df['tweets'] = filtered_df['tweets'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word, pos=wordnet.VERB) for word in x.split()]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [41]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(filtered_df.drop('dialect', axis=1), filtered_df['dialect'])
filtered_df = pd.concat([X_resampled, y_resampled], axis=1)

In [42]:
from collections import Counter
import nltk
nltk.download('punkt')
freq_dict = {"LY": Counter(), "TN": Counter(), "MA": Counter(), "DZ": Counter()}
for index, row in filtered_df.iterrows():
    dialect = row["dialect"] # Assume that "dialect" is the column in your dataset that contains the dialect label
    text = row["tweets"]
    tokens = nltk.word_tokenize(text)
    freq_dict[dialect].update(tokens)

# Print the results
for dialect, freq in freq_dict.items():
    print(f"Term frequency for {dialect}:")
    print(freq.most_common(50)) # Print the top 10 most frequent terms

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Term frequency for LY:
[('اللي', 6165), ('مش', 4043), ('الله', 3380), ('علي', 2193), ('انا', 1841), ('والله', 1749), ('ربي', 1610), ('ان', 1492), ('شن', 1308), ('شي', 1271), ('انت', 1234), ('ليبيا', 1164), ('توا', 1111), ('زي', 1078), ('حد', 987), ('هكي', 935), ('وين', 886), ('الي', 850), ('الناس', 820), ('او', 800), ('حاجه', 786), ('يارب', 781), ('يوم', 768), ('اليوم', 720), ('اي', 665), ('شاء', 645), ('يعني', 615), ('انه', 602), ('الا', 595), ('مره', 567), ('فيك', 562), ('طرابلس', 544), ('لان', 533), ('بكل', 531), ('خير', 506), ('اني', 503), ('عندي', 502), ('ممكن', 497), ('عليها', 483), ('فيهم', 482), ('عشان', 481), ('بالله', 477), ('ناس', 467), ('يلي', 463), ('بنغازي', 448), ('حتي', 445), ('احلي', 444), ('كنت', 444), ('وانت', 442), ('هلبا', 426)]
Term frequency for TN:
[('اللي', 4780), ('الله', 3832), ('مش', 2750), ('الي', 2540), ('باش', 2503), ('تونس', 2448), ('انا', 2399), ('ربي', 1655), ('والله', 1595), ('برشا', 1503), ('انت', 1327), ('ان', 1223), ('نسيم', 1210), ('موش', 1127), (

In [ ]:
from collections import Counter
import nltk
nltk.download('punkt')

# Define the maximum number of most common words to remove
num_common_words = 10

# Create the frequency dictionary
freq_dict = {"LY": Counter(), "TN": Counter(), "MA": Counter(), "DZ": Counter()}
for index, row in filtered_df.iterrows():
    dialect = row["dialect"]
    text = row["tweets"]
    tokens = nltk.word_tokenize(text)
    freq_dict[dialect].update(tokens)

# Remove the most common words from each dialect's frequency dictionary
for dialect, freq in freq_dict.items():
    print(f"Term frequency for {dialect}:")
    common_words = freq.most_common(num_common_words)
    print(f"Most common words: {common_words}")
    for word, count in common_words:
        del freq[word]

    # Print the top 10 most frequent terms after removing the common words
    print("Top terms after removing common words:")
    print(freq.most_common(10))


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, vstack
import pandas as pd

# Define the TF-IDF vectorizer and set max_features to limit the vocabulary size
tfidf = TfidfVectorizer(max_features=1000)

# Group the filtered dataframe by dialect
grouped_df = filtered_df.groupby('dialect')

# Apply TF-IDF on each group separately and store the result as a sparse matrix
tfidf_matrices = []
for dialect, group in grouped_df:
    # Fit the TF-IDF vectorizer on the text data of the current group
    tfidf.fit(group['tweets'])
    
    # Transform the text data to TF-IDF features and convert to a CSR sparse matrix
    tfidf_features = csr_matrix(tfidf.transform(group['tweets']))
    
    # Store the sparse matrix along with the dialect label
    tfidf_matrices.append((dialect, tfidf_features))
    
# Concatenate the sparse matrices into a single sparse matrix by rows
tfidf_sparse = vstack([m[1] for m in tfidf_matrices])

# Create a list of dialect labels corresponding to the rows of the sparse matrix
dialects = pd.concat([grouped_df.get_group(m[0]) for m in tfidf_matrices])['dialect'].tolist()

# Convert the sparse matrix to a dataframe and add the dialect column
tfidf_df = pd.DataFrame.sparse.from_spmatrix(tfidf_sparse, columns=tfidf.get_feature_names_out())
tfidf_df['dialect'] = dialects





NameError: ignored

In [ ]:
import seaborn as sns

# Group the TF-IDF dataframe by dialect and calculate the mean TF-IDF score for each term
tfidf_means = tfidf_df.groupby('dialect').mean()

# Sort the terms in descending order of mean TF-IDF score
tfidf_means = tfidf_means.apply(lambda x: -x).mean(axis=0).sort_values().to_frame().T


# Create a heatmap of the top 50 TF-IDF terms for each dialect
sns.heatmap(tfidf_means.iloc[:, :50], cmap='coolwarm', annot=False)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Create a dictionary to store the term frequency information for each dialect
term_freq_dict = {}

# Loop through each dialect in the filtered dataset
for dialect in filtered_df['dialect'].unique():
    
    # Subset the DataFrame for the current dialect
    dialect_df = filtered_df[filtered_df['dialect'] == dialect]
    
    # Vectorize the text data using CountVectorizer
    cv = CountVectorizer()
    cv_matrix = cv.fit_transform(dialect_df['tweets'])
    
    # Get the feature names from the CountVectorizer object
    feature_names = cv.get_feature_names()
    
    # Create a DataFrame to store the term frequency information
    term_freq_df = pd.DataFrame(cv_matrix.toarray(), columns=feature_names)
    
    # Calculate the total frequency of each term
    freq = term_freq_df.sum(axis=0)
    
    # Sort the terms by frequency
    freq_sorted = freq.sort_values(ascending=False)
    
    # Store the frequency information in the dictionary
    term_freq_dict[dialect] = freq_sorted
    
# Print the top 20 terms for each dialect
for dialect, freq_sorted in term_freq_dict.items():
    print(dialect)
    print(freq_sorted[:20])
    print()



In [ ]:
# Load data
features = filtered_df.tweets.values
labels = pd.get_dummies(filtered_df['dialect']).values

In [ ]:
vocab_size = 20000
max_length= 200
tokenizer = Tokenizer(num_words=vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(features)
X = tokenizer.texts_to_sequences(features)

In [ ]:
import nltk
from collections import Counter
nltk.download('stopwords')
from nltk.corpus import stopwords

# use the stopwords
stop_words = set(stopwords.words('arabic'))

In [ ]:
all_words = []
for tokens in X:
    all_words.extend(tokens)

stop_words = set(stopwords.words('arabic'))
all_words = [word for word in all_words if word not in stop_words]

word_counts = Counter(all_words)
most_common_words = [word for word, count in word_counts.most_common(500)]

def remove_common_words(tokens):
  new_tokens = [token for token in tokens if token not in most_common_words]
  return new_tokens
X = remove_common_words(X)
X = pad_sequences(X, maxlen=max_length)

In [ ]:
# Split data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.1, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, shuffle=True)

In [ ]:
# Convert data to PyTorch tensors
X_train_, y_train_ = torch.tensor(X_train), torch.tensor(y_train)
X_val, y_val = torch.tensor(X_val), torch.tensor(y_val)
X_test, y_test = torch.tensor(X_test), torch.tensor(y_test)

In [ ]:
class DialectDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
# Create dataloaders for training, validation, and test sets
train_dataset = DialectDataset(X_train_, y_train_ )
val_dataset = DialectDataset(X_val, y_val)
test_dataset = DialectDataset(X_test, y_test)

In [ ]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import numpy as np

vocab_size = 20000
embedding_dim = 100
hidden_dim = 128
output_dim = 4
num_layers = 2
bidirectional = True
lr = 0.01
batch_size = 64
num_epochs = 11

class BiLSTMAttention(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers):
        super(BiLSTMAttention, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=True, dropout=0.5, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(0.5)
        self.attention = SelfAttention(hidden_dim * 2)
        
    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = self.dropout(lstm_out)
        attention_out, attention_weights = self.attention(lstm_out)
        fc_out = self.fc(attention_out)
        return fc_out, attention_weights

class SelfAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(SelfAttention, self).__init__()
        self.projection = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )
        
    def forward(self, encoder_outputs):
        energy = self.projection(encoder_outputs)
        weights = F.softmax(energy.squeeze(-1), dim=1)
        outputs = (encoder_outputs * weights.unsqueeze(-1)).sum(dim=1)
        return outputs, weights

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTMAttention(vocab_size, embedding_dim, hidden_dim, output_dim, num_layers).to(device)
optimizer = optim.Adam(model.parameters(),lr=0.01)
criterion = nn.BCEWithLogitsLoss()

for epoch in tqdm(range(num_epochs)):
     model.train()
     running_loss = 0.0
     with torch.cuda.device(0):
       for batch in train_dataloader:
         inputs, labels = batch[0].to(device), batch[1].to(device)
         optimizer.zero_grad()
         labels = labels.float()
         outputs, _ = model(inputs)
         loss = criterion(outputs, labels)
         loss.backward()
         optimizer.step()
         running_loss += loss.item()
         preds = torch.sigmoid(outputs) > 0.5
         acc = accuracy_score(labels.cpu().detach().numpy(), preds.cpu().detach().numpy())
         #print('Train Accuracy: {:.4f}'.format(acc))
     epoch_loss = running_loss / (len(train_dataloader))

     val_loss = 0.0
     model.eval()
     with torch.no_grad():
         for batch in val_dataloader:
             inputs, labels = batch[0].to(device), batch[1].to(device)
             labels = labels.float()
             outputs, _= model(inputs)
             loss = criterion(outputs, labels)
             val_loss += loss.item()
             #val_preds = torch.sigmoid(outputs) > 0.5
             #val_acc = accuracy_score(labels.cpu().detach().numpy(), val_preds.cpu().detach().numpy())
             #print('Val Accuracy: {:.4f}'.format(val_acc))
     val_loss = val_loss / (len(val_dataloader))
     
     print(" epoch loss :", epoch_loss , "| val loss :", val_loss)
     #print('Train Accuracy: {:.4f}'.format(acc))
     #print('Val Accuracy: {:.4f}'.format(val_acc))

print("Training stopped after epoch", epoch)
        

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for batch in test_dataloader:
        inputs, labels = batch[0].to(device), batch[1].to(device)
        labels = labels.float()
        outputs, _ = model(inputs)
        preds = torch.sigmoid(outputs) > 0.5
        y_true.extend(labels.cpu().detach().numpy())
        y_pred.extend(preds.cpu().detach().numpy())
test_acc = accuracy_score(y_true, y_pred)
test_f1 = f1_score(y_true, y_pred, average='macro')
test_pres = precision_score(y_true, y_pred, average='macro')
test_recall = recall_score(y_true, y_pred, average='macro')
print('Test Accuracy: {:.4f}'.format(test_acc))
print('Test f1 score: {:.4f}'.format(test_f1))
print('Test precision: {:.4f}'.format(test_pres))
print('Test recall: {:.4f}'.format(test_recall))

In [ ]:
# Set the model to evaluation mode
model.eval()
device = "cuda"
# Tokenize the new complaint and pad the sequence
#new_complaint = ['واش دير للعشى']
#new_complaint  = ["شوكران علا هاد "]
#new_complaint  = ["نبي نروح للحوش"]
new_complaint  = ["شبيك شتحب "]
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=max_length)

# Convert the padded sequence to a PyTorch tensor and move it to the device (e.g., GPU) if available
padded_tensor = torch.LongTensor(padded).to(device)

# Compute the model's prediction for the padded sequence
with torch.no_grad():
    pred = model(padded_tensor)

# Move the prediction back to the CPU and convert to a numpy array
#pred = pred.cpu().numpy()

# Map the prediction to a class label using the CLASS_DICT{'DZ': 0, 'LY': 1, 'MA': 2, 'TN': 3}
CLASS_DICT = {0: "DZ",1: "LY", 2: "MA",3: "TN"}
class_label = CLASS_DICT[np.argmax(pred[0].cpu().numpy(), axis=1)[0]]

# Print the prediction and the predicted class label
print(class_label)

In [ ]:
torch.save(model.state_dict(), 'model3.pth')